# Training locally with a GPU

To train locally, run this notebook.

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset
from transformers import Trainer, TrainingArguments

device = torch.device("cuda")

dataset = load_dataset('json', data_files={'train': '../data/train/t5_qa_pairs.jsonl'})

model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base').to(device)
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base')

def preprocess_function(examples):
    inputs = examples['prompt']
    targets = examples['completion']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length')

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

training_args = TrainingArguments(
    output_dir='./flan-t5-model',
    num_train_epochs=3,
    per_device_train_batch_size=8
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
)

trainer.train()

model.save_pretrained('./flan-t5-model')
tokenizer.save_pretrained('./flan-t5-model')

In [ ]:
sagemaker_session.upload_data("./flan-t5-model-local-train", key_prefix="flan-t5-environment-ft")